In [1]:
%load_ext autoreload

In [12]:
%autoreload 2
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import yaml
import json
import os
from flatten_dict import unflatten, flatten
from filter_utils import *
from pandas.api.types import is_numeric_dtype
import seaborn as sns
from sklearn.preprocessing import minmax_scale, LabelEncoder




import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())
os.getcwd()

Number of processors:  8


'/Users/morarica/Developer/SepMe/Jupyters'

## 1. Getting the human annotations

In [4]:
def underscore_reducer(k1, k2):
    if k1 is None:
        return str(k2)
    else:
        return str(k1) + "_" + str(k2)

In [5]:
folder = '/Users/morarica/Developer/SepMe/data/SPLOM_data/SepMe/Data/Human_judgment/'
print(len(sorted(os.listdir(folder))))
df_a = []
df_m = []
for i, file in enumerate(sorted(os.listdir(folder))):
    if file.endswith('csv'):
        #print(file)
        df = pd.read_csv(folder+file, header=None)
        df.columns = np.array(df.columns)+1
        
        if len(df)>4:
            #print(df)
            df = df.head(4)
        
        df.index = ['PCA', 'RobPCA', 'GlimmerMDS', 'tSNE']
        
        person = file.split('.csv')[0].split('-')[-1]
        dd ={}
        dd[file.split('-')[0]] = df.to_dict('index')
        df1 = pd.DataFrame.from_dict(flatten(dd, underscore_reducer), orient='index', columns = [person])
        
        #print(person + '  ' + file)
        if person =='M':
            df_m.append(df1)
        else:
            df_a.append(df1)

dfmm = pd.concat(df_m, axis=0)
dfaa = pd.concat(df_a, axis=0)

df = pd.concat([dfmm, dfaa], axis=1)


df['class'] = [ii.split('_')[-1] for ii in df.index]
df.dropna(axis=0, inplace=True)

df = df.astype({'A': 'int32', 'M': 'int32'})
print(df.shape)

153
(1669, 3)


In [6]:
df['dim_x'] = 1
df['dim_y'] = 2
df = df[df.A.between(1,5) & df.M.between(1,5)]
df = df.reset_index()
print(df.shape)
df.head()


names = ['_'.join(row.split('_')[:-1]) for row in list(df['index'])]
#names = list(set(names))

df['fileName'] = names

(1648, 6)


In [7]:
df.to_csv('/Users/morarica/Developer/SepMe/data/orig_data/human_reduced_results.csv', index=False)

## 2. Plot inputs

In [8]:
def remove_outlier(df, dims = [.05,.95]):
    low = dims[0]
    high = dims[1]
    quant_df = df.quantile([low, high])
    #print(quant_df)
    for name in list(df.columns):
        if name in ['x','y']:
            df = df[(df[name] > quant_df.loc[low, name]) 
               & (df[name] < quant_df.loc[high, name])]
        #print(len(df))
    return df

def preprocess_df(df, dims, size = 0.5, occlusion = 0.1, save=None, sort=True):
    
    ddf = df[dims + ['class']].copy()
    #print(ddf.head())
    
    #print(ddf.shape)
    classes = list(set(ddf['class']))
    if len(classes)>10:
        print('Classes: {}'.format(len(classes)))
        ddf.columns = ['x', 'y', 'orig_class'] #rename cols
        merge_classes = list(ddf[['orig_class', 'x']].groupby(['orig_class']).count()
                             .sort_values(['x'], ascending=True).head(len(classes)-9).index)
        
        ddf['class'] = [row if row not in merge_classes else -1 for i,row in ddf['orig_class'].items()]
        
        le = LabelEncoder()
        ddf['class'] = le.fit_transform(ddf['class'])
        
    else: 
        ddf.columns = ['x', 'y', 'class'] #rename cols
        le = LabelEncoder()
        ddf['orig_class'] = ddf['class']
        ddf['class'] = le.fit_transform(ddf['orig_class'])
        
        
    ddf = remove_outlier(ddf, [0.05, 0.95]) #remove outliers
    ddf.iloc[:,:2] = minmax_scale(ddf.iloc[:, :2]) #scale between 0 and 1
    
    if sort == True:
        ddf = ddf.sort_values(['x', 'y']) # sort
        
    #print(ddf.shape)   
    
    ddf.reset_index(inplace=True, drop=True) #reset_index 
    
    # filter occluded circles
    buff = ((max(ddf.x) - min(ddf.x)) / (np.sqrt(len(ddf.x))*2))*size
    circles = get_circles(ddf, buff)
    rem_indexes = remove_circles_by_partition(ddf, circles, 2*buff, occlusion)
    ddf = ddf.drop(rem_indexes)
    circles = circles.drop(rem_indexes)
    
    if save is not None:
        ddf.to_csv(save, index=False)
    
    return ddf, circles


def plot_colored_circles(ax, df, circles, class_name = 'class'):
    
    for i, row in df.iterrows():
        ax.add_patch(PolygonPatch(circles[i], fc = sns.color_palette("colorblind")[int(row[class_name])], ec = 'none', alpha = 1))
            
    ax.autoscale()
    ax.set_aspect('equal', 'datalim')

    return ax

def get_dimred_data(df, input_folder, save_folder, fig_folder):
    found = 0
    not_found = []
    names = ['_'.join(row.split('_')[:-1]) for row in list(df['index'])]
    names = list(set(names))
    
    for i, nn in enumerate(names):
        
        print('Processing file {}/{}. Name: {}'.format(i, len(names), nn))
        
        method = nn.split('_')[-1]
        #print(method)
        if method in {'PCA', 'RobPCA'}:
            name = nn + '_data.csv'
        else: 
            name = nn + '_2.csv'
            
        #print(name)
            
        try:
            data = pd.read_csv(input_folder+method+'/'+name)
            found += 1
            
            dim_x = str(df.loc[i, 'dim_x'])
            dim_y = str(df.loc[i, 'dim_y'])
            
            file_name = save_folder + nn + '_' + dim_x +'-' + dim_y +'.csv'
            
            if os.path.exists(file_name) is True:
                print('File was already processed. Skipping: {}'.format(file_name))
                continue

            ddf, circles = preprocess_df(data, [dim_x, dim_y], size = 0.5, occlusion = 0.1, save=file_name, sort=True)


            fig, ax = plt.subplots(figsize = (10,10))
            plot_colored_circles(ax, ddf, circles)
            fig.savefig(fig_folder + names[i] + '.pdf')
            plt.close('all')


        except FileNotFoundError:
            #print(nn)
            print('File not found: {}'.format(name))
            not_found.append(name)
            
    return 



In [9]:
input_folder = '/Users/morarica/Developer/SepMe/data/orig_data/input_data/Reduced_orig_data/'
save_folder = '/Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/'
fig_folder = '/Users/morarica/Developer/SepMe/data/orig_data/figures/reduced_data/'

get_dimred_data(df, input_folder, save_folder, fig_folder)

Processing file 0/216. Name: boston_GlimmerMDS
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/boston_GlimmerMDS_1-2.csv
Processing file 1/216. Name: bbdm13_GlimmerMDS
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/bbdm13_GlimmerMDS_1-2.csv
Processing file 2/216. Name: cars03Cropped_d3_RobPCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/cars03Cropped_d3_RobPCA_1-2.csv
Processing file 3/216. Name: ms_interleaved_120_240_3d_25_centeredClusters_tSNE
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/ms_interleaved_120_240_3d_25_centeredClusters_tSNE_1-2.csv
Processing file 4/216. Name: fisheries_clusteredByHarvestRule_PCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/fisheries_clus

File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/ecoliproteins_GlimmerMDS_1-2.csv
Processing file 46/216. Name: ms_interleaved_60_120_3d_0_centeredClusters_PCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/ms_interleaved_60_120_3d_0_centeredClusters_PCA_1-2.csv
Processing file 47/216. Name: interleaved_100_200_4d_0_notcramped_notrotated_tSNE
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/interleaved_100_200_4d_0_notcramped_notrotated_tSNE_1-2.csv
Processing file 48/216. Name: interleaved_100_200_4d_25_cramped_rotated_RobPCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/interleaved_100_200_4d_25_cramped_rotated_RobPCA_1-2.csv
Processing file 49/216. Name: iris_tSNE
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/

File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/cars03Cropped_d1_PCA_1-2.csv
Processing file 110/216. Name: n100_PCA
File not found: n100_PCA_data.csv
Processing file 111/216. Name: grid10_3d_GlimmerMDS
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/grid10_3d_GlimmerMDS_1-2.csv
Processing file 112/216. Name: hiv_GlimmerMDS
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/hiv_GlimmerMDS_1-2.csv
Processing file 113/216. Name: ms_interleaved_60_120_3d_25_centeredClusters_tSNE
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/ms_interleaved_60_120_3d_25_centeredClusters_tSNE_1-2.csv
Processing file 114/216. Name: white_ballance_RobPCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/white

File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/shuttle_big_PCA_1-2.csv
Processing file 165/216. Name: ms_interleaved_40_80_3d_0_GlimmerMDS
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/ms_interleaved_40_80_3d_0_GlimmerMDS_1-2.csv
Processing file 166/216. Name: spambase_PCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/spambase_PCA_1-2.csv
Processing file 167/216. Name: efashion_GlimmerMDS
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/efashion_GlimmerMDS_1-2.csv
Processing file 168/216. Name: ms_interleaved_60_120_3d_25_centeredClusters_PCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/ms_interleaved_60_120_3d_25_centeredClusters_PCA_1-2.csv
Processing file 169/216. Name: f

File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/cars03Cropped_d1_RobPCA_1-2.csv
Processing file 215/216. Name: grid10_3d_PCA
File was already processed. Skipping: /Users/morarica/Developer/SepMe/data/orig_data/input_data/reduced_clean/grid10_3d_PCA_1-2.csv


## 3. Plot dimensions in original data

In [10]:
def save_and_plot_all_dimensions(orig_dir, save_dir, fig_dir, class_cols):
    for i, file in enumerate(os.listdir(orig_dir)):
        if file.endswith('.csv'):
            print('Processing file {}/{}. Name: {}'.format(i, len(os.listdir(orig_dir)), file))
            if file in ['shuttle_small_origClassLabels.csv', 'shuttle_big.csv']:
                continue
            
            df = pd.read_csv(orig_dir + file)

            c1s = []
            for c1 in df.columns:
                if not np.issubdtype(df[c1].dtype, np.number) or (c1 in class_cols):
                    continue

                c1s.append(c1)
                for c2 in df.columns[:-1]:
                    if (not np.issubdtype(df[c1].dtype, np.number)) or (c2 in c1s) or (c2 in class_cols):
                        continue

                    file_name = save_dir + file.split('.csv')[0] + '_{}-{}'.format(c1,c2) + '.csv'
                    fig_name = fig_dir + file.split('.csv')[0] + '_{}-{}'.format(c1,c2) + '.pdf'
                    
                    if os.path.exists(file_name) is True:
                        print('File was already processed. Skipping: {}'.format(file_name.split('/')[-1]))
                        continue

                    try: 
                        print('{}-{}'.format(c1,c2))
                        ddf, circles = preprocess_df(df, [c1, c2], size = 0.5, 
                                                 occlusion = 0.1, save=file_name, sort=True)
                        fig, ax = plt.subplots(figsize = (10,10))
                        plot_colored_circles(ax, ddf, circles)
                        fig.savefig(fig_name)
                        plt.close('all')
                        
                    except Exception as e: 
                        print('File {} was not processed for columns ({},{}).'.format(file,c1,c2))
                        print(e)
                        print('')


In [11]:
orig_dir = "/Users/morarica/Developer/SepMe/data/orig_data/input_data/Orig_data/"
save_dir = "/Users/morarica/Developer/SepMe/data/orig_data/input_data/dims_clean/"
fig_dir = "/Users/morarica/Developer/SepMe/data/orig_data/figures/orig_dim_data/"
class_cols = ['class']

save_and_plot_all_dimensions(orig_dir, save_dir, fig_dir, class_cols)

Processing file 0/104. Name: grid10_3d.csv
File was already processed. Skipping: grid10_3d_d1-d2.csv
File was already processed. Skipping: grid10_3d_d1-d3.csv
File was already processed. Skipping: grid10_3d_d2-d3.csv
Processing file 1/104. Name: interleaved_100_200_4d_0_notcramped_notrotated.csv
File was already processed. Skipping: interleaved_100_200_4d_0_notcramped_notrotated_d1-d2.csv
File was already processed. Skipping: interleaved_100_200_4d_0_notcramped_notrotated_d1-d3.csv
File was already processed. Skipping: interleaved_100_200_4d_0_notcramped_notrotated_d1-d4.csv
File was already processed. Skipping: interleaved_100_200_4d_0_notcramped_notrotated_d2-d3.csv
File was already processed. Skipping: interleaved_100_200_4d_0_notcramped_notrotated_d2-d4.csv
File was already processed. Skipping: interleaved_100_200_4d_0_notcramped_notrotated_d3-d4.csv
Processing file 2/104. Name: abalone.csv
File was already processed. Skipping: abalone_x1-x2.csv
File was already processed. Skipping

File ionosphere.csv was not processed for columns (D2,D4).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D2-D5
File ionosphere.csv was not processed for columns (D2,D5).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D2-D6
File ionosphere.csv was not processed for columns (D2,D6).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D2-D7
File ionosphere.csv was not processed for columns (D2,D7).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D2-D8
File ionosphere.csv was not processed for columns (D2,D8).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D2-D9
File ionosphere.csv was not processed for columns (D2,D9).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D2-D10
File ionosphere.csv was not processed for columns (D2,D10).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is require

File was already processed. Skipping: ionosphere_D14-D30.csv
File was already processed. Skipping: ionosphere_D14-D31.csv
File was already processed. Skipping: ionosphere_D14-D32.csv
File was already processed. Skipping: ionosphere_D14-D33.csv
File was already processed. Skipping: ionosphere_D14-D34.csv
File was already processed. Skipping: ionosphere_D15-D16.csv
File was already processed. Skipping: ionosphere_D15-D17.csv
File was already processed. Skipping: ionosphere_D15-D18.csv
File was already processed. Skipping: ionosphere_D15-D19.csv
File was already processed. Skipping: ionosphere_D15-D20.csv
File was already processed. Skipping: ionosphere_D15-D21.csv
File was already processed. Skipping: ionosphere_D15-D22.csv
File was already processed. Skipping: ionosphere_D15-D23.csv
File was already processed. Skipping: ionosphere_D15-D24.csv
File was already processed. Skipping: ionosphere_D15-D25.csv
File was already processed. Skipping: ionosphere_D15-D26.csv
File was already process

File bbdm13_origClassLabels.csv was not processed for columns (V10,V12).
float division by zero

V10-V13
File bbdm13_origClassLabels.csv was not processed for columns (V10,V13).
float division by zero

File was already processed. Skipping: bbdm13_origClassLabels_V11-V12.csv
File was already processed. Skipping: bbdm13_origClassLabels_V11-V13.csv
File was already processed. Skipping: bbdm13_origClassLabels_V12-V13.csv
Processing file 9/104. Name: interleaved_100_200_5d_25_cramped_rotated.csv
File was already processed. Skipping: interleaved_100_200_5d_25_cramped_rotated_d1-d2.csv
File was already processed. Skipping: interleaved_100_200_5d_25_cramped_rotated_d1-d3.csv
File was already processed. Skipping: interleaved_100_200_5d_25_cramped_rotated_d1-d4.csv
File was already processed. Skipping: interleaved_100_200_5d_25_cramped_rotated_d1-d5.csv
File was already processed. Skipping: interleaved_100_200_5d_25_cramped_rotated_d2-d3.csv
File was already processed. Skipping: interleaved_100_

File ionosphere_origClassLabels.csv was not processed for columns (D1,D27).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D1-D28
File ionosphere_origClassLabels.csv was not processed for columns (D1,D28).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D1-D29
File ionosphere_origClassLabels.csv was not processed for columns (D1,D29).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D1-D30
File ionosphere_origClassLabels.csv was not processed for columns (D1,D30).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D1-D31
File ionosphere_origClassLabels.csv was not processed for columns (D1,D31).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D1-D32
File ionosphere_origClassLabels.csv was not processed for columns (D1,D32).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

D1-D33
File ionosphere_origClassLab

...

year-d2
File efashion_origClassLabels.csv was not processed for columns (year,d2).
float division by zero

year-d3
File efashion_origClassLabels.csv was not processed for columns (year,d3).
float division by zero

year-d4
File efashion_origClassLabels.csv was not processed for columns (year,d4).
float division by zero

File was already processed. Skipping: efashion_origClassLabels_d2-d3.csv
File was already processed. Skipping: efashion_origClassLabels_d2-d4.csv
File was already processed. Skipping: efashion_origClassLabels_d3-d4.csv
Processing file 51/104. Name: white_ballance_origClassLabels.csv
File was already processed. Skipping: white_ballance_origClassLabels_height-lengthe.csv
File was already processed. Skipping: white_ballance_origClassLabels_height-area.csv
File was already processed. Skipping: white_ballance_origClassLabels_height-eccen.csv
File was already processed. Skipping: white_ballance_origClassLabels_height-p_black.csv
File was already processed. Skipping: white_ball

File cereal.csv was not processed for columns (Fibre,Display_shelf).
float division by zero

Fibre-Potassium
File cereal.csv was not processed for columns (Fibre,Potassium).
float division by zero

File was already processed. Skipping: cereal_Complex_carbohydrates-Sugars.csv
File was already processed. Skipping: cereal_Complex_carbohydrates-Display_shelf.csv
File was already processed. Skipping: cereal_Complex_carbohydrates-Potassium.csv
Sugars-Display_shelf
File cereal.csv was not processed for columns (Sugars,Display_shelf).
float division by zero

Sugars-Potassium
File cereal.csv was not processed for columns (Sugars,Potassium).
float division by zero

File was already processed. Skipping: cereal_Display_shelf-Potassium.csv
Processing file 69/104. Name: cars03Cropped_d3.csv
d1-d2
Classes: 12
File cars03Cropped_d3.csv was not processed for columns (d1,d2).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

d1-d3
Classes: 12
File cars03Cropped_d3.csv was no

Processing file 75/104. Name: world_11d.csv
File was already processed. Skipping: world_11d_d1-d2.csv
File was already processed. Skipping: world_11d_d1-d3.csv
File was already processed. Skipping: world_11d_d1-d4.csv
File was already processed. Skipping: world_11d_d1-d5.csv
File was already processed. Skipping: world_11d_d1-d6.csv
File was already processed. Skipping: world_11d_d1-d7.csv
File was already processed. Skipping: world_11d_d1-d8.csv
File was already processed. Skipping: world_11d_d1-d9.csv
File was already processed. Skipping: world_11d_d1-d10.csv
File was already processed. Skipping: world_11d_d1-d11.csv
File was already processed. Skipping: world_11d_d1-d12.csv
File was already processed. Skipping: world_11d_d2-d3.csv
File was already processed. Skipping: world_11d_d2-d4.csv
File was already processed. Skipping: world_11d_d2-d5.csv
File was already processed. Skipping: world_11d_d2-d6.csv
File was already processed. Skipping: world_11d_d2-d7.csv
File was already processe

File cars03Cropped_d2.csv was not processed for columns (d1,d22).
Found array with 0 sample(s) (shape=(0, 2)) while a minimum of 1 is required.

File was already processed. Skipping: cars03Cropped_d2_d2-d3.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d4.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d5.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d6.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d7.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d8.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d9.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d19.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d11.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d12.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d13.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d14.csv
File was already processed. Skipping: cars03Cropped_d2_d2-d15.csv
File

KeyboardInterrupt: 

In [ ]:
pd.read_csv()